In [93]:
import os
%pwd



'/home/cs/Chicken-Disease-Classification'

In [94]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
  root_dir: Path
  source_URL: str
  local_data_file: Path
  unzip_dir: Path


In [95]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directory


In [96]:

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        print(self.config.keys())  # This will print all keys in the Box object

        create_directory([self.config.artifacts_root])

    def get_data_ingestion_config(self):
        config = self.config.data_ingestion

        create_directory([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [97]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [98]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename,header=request.urlretrieve(url=self.config.source_URL,filename=self.config.local_data_file)
            logger.info(f'Downloaded {filename} with info:{header}')
        else:
            logger.info(f'file already exists with size:{get_size(Path(self.config.local_data_file))}')


    def extract_zip_file(self):

        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_file:
            zip_file.extractall(unzip_path)

In [99]:

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-07 23:56:22,650: INFO: common:Reading yaml file from:config/config.yaml]
[2024-12-07 23:56:22,653: INFO: common:Reading yaml file from:params.yaml]


dict_keys(['artifacts_root', 'data_ingestion'])


AttributeError: 'Logger' object has no attribute 'inf'